## Notebook for making predictions with deleted constant y samples


In [25]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [26]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [27]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [28]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

catModel_bc = CatBoostRegressor()

#catModel_abc = CatBoostRegressor()

In [29]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

X_ABC_train = pd.concat([X_A_train, X_BC_train])
Y_ABC_train = pd.concat([Y_A_train, Y_BC_train])

In [30]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)

#catModel_abc.fit(X_ABC_train,Y_ABC_train)

Learning rate set to 0.067297
0:	learn: 1247.3873765	total: 15.3ms	remaining: 15.3s
1:	learn: 1185.3866455	total: 26.7ms	remaining: 13.3s
2:	learn: 1128.9035875	total: 36.8ms	remaining: 12.2s
3:	learn: 1075.9568829	total: 48.6ms	remaining: 12.1s
4:	learn: 1027.6909816	total: 59.2ms	remaining: 11.8s
5:	learn: 984.1590189	total: 68.8ms	remaining: 11.4s
6:	learn: 943.3631632	total: 78.7ms	remaining: 11.2s
7:	learn: 906.3431773	total: 88ms	remaining: 10.9s
8:	learn: 870.8696255	total: 99.7ms	remaining: 11s
9:	learn: 839.4499830	total: 110ms	remaining: 10.9s
10:	learn: 811.1590205	total: 121ms	remaining: 10.9s
11:	learn: 784.4830023	total: 132ms	remaining: 10.9s
12:	learn: 761.1293361	total: 143ms	remaining: 10.8s
13:	learn: 739.3893761	total: 153ms	remaining: 10.8s
14:	learn: 719.6525654	total: 165ms	remaining: 10.8s
15:	learn: 701.3762672	total: 177ms	remaining: 10.9s
16:	learn: 685.1435456	total: 189ms	remaining: 10.9s
17:	learn: 669.9784557	total: 201ms	remaining: 11s
18:	learn: 656.461

In [31]:
# Feature importance
catModel_a.get_feature_importance(prettified=True)

,Feature Id,Importances
0,direct_rad:W,25.809874
1,sum_rad:W,11.386710
2,diffuse_rad:W,6.258336
3,sun_azimuth:d,4.380955
4,day_year,4.133488
5,clear_sky_rad:W,3.897358
6,direct_rad_1h:J,3.882433
7,wind_speed_u_10m:ms,3.686527
8,sun_elevation:d,2.813629
9,diffuse_rad_1h:J,2.596775


## Preds

In [32]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

# pred_B = catModel_bc.predict(X_B_test)
# pred_C = catModel_bc.predict(X_C_test)

# pred_A = catModel_abc.predict(X_A_test)
# pred_B = catModel_abc.predict(X_B_test)
# pred_C = catModel_abc.predict(X_C_test)

In [33]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/catboost_prelag_25.csv")

df.shape

(2160, 1)